# probability of the real structure,  under different SHAPE arrays

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
from eden.util import configure_logging
import logging
configure_logging(logging.getLogger(),verbosity=3)
import sys
sys.path.insert(0,'..')
from IPython.core.display import HTML
HTML('<style>.container { width:95% !important; }</style><style>.output_png {display: table-cell;text-align: center;vertical-align: middle;}</style>')

# Load data

In [2]:
import rna_tools.rna_io as rio

def getdata():
    return rio.get_all_data("../data/RNA16.react","../data/RNA16.dbn")  # {key: rea, seq, stru}

data = getdata()

for k,v in data.items():print k,"\t",  len(v[1])

data for 'tRNA-asp' is corrupted, ignoring...
ADDRSW 	121
GLYCFN 	205
tRNAphe 	76
RNAsep 	154
ZHCV 	95
Z-CIDGMP-1 	135
adenin 	71
23sRNA 	2904
p564 	155
srRNA 	120
ZGLY 	198
MDLOOP 	80
R009 	1542
5srRNA 	170
TRP5 	202


# train predict

In [3]:
'''
import simushape as sim
from rna_tools.rnaplfold import rnaplfold

data = getdata()
model  = sim.make_model(data,['23sRNA','R009'])
#data.pop("23sRNA")
#data.pop("R009")

predictions = { k: (v[1],sim.predict(model,v[1])) for k,v in data.items()[:15]  }

keys = predictions.keys()[:15]


acc_shaker = {k: rnaplfold(*predictions[k]) for k in keys} # a predcition is seq,rea # probability

'''

'\nimport simushape as sim\nfrom rna_tools.rnaplfold import rnaplfold\n\ndata = getdata()\nmodel  = sim.make_model(data,[\'23sRNA\',\'R009\'])\n#data.pop("23sRNA")\n#data.pop("R009")\n\npredictions = { k: (v[1],sim.predict(model,v[1])) for k,v in data.items()[:15]  }\n\nkeys = predictions.keys()[:15]\n\n\nacc_shaker = {k: rnaplfold(*predictions[k]) for k in keys} # a predcition is seq,rea # probability\n\n'

# real and sukosd based accessibility

In [4]:
from rna_tools.sukosd import sukosd
from rna_tools.structureprobability import probabilities_of_structures
import simushape as sim

data = getdata()
model  = sim.make_model(data,['23sRNA','R009'])
predictions = { k: (v[1],sim.predict(model,v[1])) for k,v in data.items()[:15]  }
keys = predictions.keys()[:15]
#acc_suko = {k: rnaplfold(data[k][1], sukosd(data[k][2]) ) for  k in keys} # sukosd returns probability (shape) based on real strcuture
#acc_shape =  {k: rnaplfold(data[k][1],data[k][0]) for  k in keys} # real shape 
prb_suko = [probabilities_of_structures(data[k][1], [data[k][2]], sukosd(data[k][2]))[0][1] for  k in keys] # input: seq and str, returns probability (sukosd(data[k][2]) returns shape)
prb_real =  [probabilities_of_structures(data[k][1],[data[k][2]], data[k][0])[0][1] for  k in keys] #rnaplfold gets sequence and shape, returns probab 
prb_shaker = [probabilities_of_structures(data[k][1],[data[k][2]], predictions[k])[0][1] for k in keys] # a predcition is seq,rea # probability




data for 'tRNA-asp' is corrupted, ignoring...


/home/montaser/miniconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/home/montaser/miniconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/home/montaser/miniconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a

# Visualize 

In [5]:
import pandas
import numpy as np
index= ['suko',"shaker", "real"]
#data = [corr_suko,corr_shaker]
data = [prb_suko, prb_real, prb_shaker] # data is a list 

map(lambda x:x.append(np.mean(x)),data)
pandas.DataFrame(data, columns=keys+['mean'], index=index).T


,suko,shaker,real
ADDRSW,4.509249e-02,1.598590e-01,2.284345e-01
GLYCFN,2.628612e-05,5.845614e-25,7.900716e-07
tRNAphe,4.589490e-01,4.818414e-01,6.137467e-02
ZHCV,4.417911e-05,8.556295e-08,2.234914e-05
Z-CIDGMP-1,6.512101e-06,2.976081e-09,9.417846e-09
adenin,1.253252e-01,5.058758e-01,2.175815e-01
23sRNA,0.000000e+00,0.000000e+00,0.000000e+00
MDLOOP,2.321712e-01,2.820767e-01,1.155595e-01
srRNA,3.096035e-04,1.010630e-04,5.174185e-08
ZGLY,3.275749e-10,1.889473e-09,6.929155e-08
